# Code to scrap tweets from twitter using search terms defined by the twitter search parameters 

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# define the search term (I'm very happy with these search terms so far)
search_term = "(#OBOR OR #BeltandRoad OR #BeltandRoadInitiative OR #OneBeltOneRoad)"
#Define the timespan to look for tweets 
since_date = "2021-01-01" #from
until_date = "2021-12-31"   #to

#7.5 min for 10k tweets + 27 min for 45k tweets
limit = 100000

# create an empty list to store the tweets
tweets_all_list = []

count = 0

# use snscrape to scrape tweets
for tweet in sntwitter.TwitterSearchScraper(f"{search_term} since:{since_date} until:{until_date}").get_items():

    # append the tweet information to the list                                                      #Languages are categorized by "ISO 639-1 codes"
    tweets_all_list.append([tweet.id, tweet.date, tweet.content, tweet.user.username, tweet.url, tweet.hashtags, tweet.lang])

    count += 1  # increment the counter variable

    if count == limit:  # break out of the loop once 100000 tweets have been scraped
        print("\nTweets have finsihed loading")
        break
    else:
        print(f"Loading Tweet Number: {count}")


# convert the list of lists into a pandas DataFrame
tweets_all = pd.DataFrame(tweets_all_list, columns=['id', 'date', 'content', 'username', 'url', 'hashtag', 'language'])

#just cleaning up the variable view 
del tweets_all_list, tweet

## Some basic investigation into the tweets dataframe to make sure it looks good

In [ ]:
# Grouping each tweet by lanuage to see how many lanauges are represented to see how many we should focus on 
re = tweets_all.groupby('language').size().reset_index(name='count').sort_values('count', ascending=False)
re

In [ ]:
#Creating a data frame tweets based on a particular language 
#tweets_filter= tweets_all[(tweets_all["language"] == "en")] 

In [ ]:
#Finds the number of tweets posted by each person 
unqiue_tweeters = tweets_all['username'].value_counts()
unqiue_tweeters

In [ ]:
sin = tweets_all[(tweets_all['username'] != "youloveBri_")]

In [ ]:
#checking all the tweets of our number 1 tweeter to see if they're all unique
only_beta = tweets_all[(tweets_all['username'] == "youloveBri_")]
seeing = only_beta['content'].value_counts()

# Doing some quick preprocessing of the dataframe before making it into an excel file and loading it into R

In [6]:
#removing the timezone from that date format automatically gathered by SNScrape (only run once)
tweets_all['date'] = tweets_all['date'].dt.tz_convert(None)

In [7]:
#converting the date to a new format changes it to a string instead of a date 
tweets_all['date'] = tweets_all['date'].dt.strftime("%b %d, %Y") 

In [ ]:
# check the data type of the date column
print(tweets_all['date'].dtype)

# Creating the excel file to be loaded in by R 

In [8]:
path = r"C:\Users\Ryan\Coding Projects\Twitter Data Scraping\\"

tweets_all.to_excel(path + 'master_tweets_2021.xlsx', index=False)